# **Generative Resume Prompt Engineering**

In [1]:
# Step 1: Install required libraries
!pip install transformers accelerate sentencepiece huggingface_hub -q

In [2]:
# Step 2: Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
from google.colab import userdata

In [3]:
# Step 3: Hugging Face authentication
HF_TOKEN = userdata.get("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("HF_TOKEN not found in Colab secrets!")

login(HF_TOKEN)

In [4]:
# Step 4: Use tiiuae/Falcon3-1B-Instruct model
model_name = "tiiuae/Falcon3-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Set common generation parameters
MAX_TOKENS = 500
TEMPERATURE = 0.6
TOP_P = 0.9

In [6]:
# Set 1 of ZERO-SHOT PROMPTING
# =====================================================
zero_shot_prompt = """You are an expert resume writer.
Create a complete, single-page professional resume for a Data Scientist with 2 years of experience.
**Invent a generic name, contact details, company name, and specific achievements.**
Include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the final output ONLY as a resume, with clear headings and bullet points.
**DO NOT include any introductory text, invented job descriptions, or notes before or after the resume.**
"""

zero_shot = generator(
    zero_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("====================================")
print("📄 ZERO-SHOT RESUME")
print("====================================\n")
print(zero_shot)

📄 ZERO-SHOT RESUME

You are an expert resume writer.
Create a complete, single-page professional resume for a Data Scientist with 2 years of experience.
**Invent a generic name, contact details, company name, and specific achievements.**
Include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the final output ONLY as a resume, with clear headings and bullet points.
**DO NOT include any introductory text, invented job descriptions, or notes before or after the resume.**
<|assistant|>
**Professional Summary:**

**Data Scientist**

With 2 years of experience in leveraging analytics and machine learning to drive impactful decision-making, I specialize in transforming raw data into actionable insights. My expertise spans Python programming, advanced machine learning algorithms, and robust statistical analysis. I excel in creating predictive models that enhance organizational efficiency and innovation.

**Skills:**
- Python

In [7]:
# Set 1 of ONE-SHOT PROMPTING
# =====================================================
one_shot_prompt = """You are an expert resume writer.

--- EXAMPLE START (SIMPLE FORMAT) ---
Software Engineer Resume
Name: Alex Johnson | Email: alex.j@mail.com
Summary: Software Engineer with 3 years of experience in full-stack development.
Skills: Java, Spring Boot, Docker, Kubernetes
Experience: Engineer at TechCo (2021–2023). Managed deployment pipelines.
Education: B.Tech in Computer Science
--- EXAMPLE END ---

TASK: Now, create a complete professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Include: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the new resume **exactly** like the simple example above.
"""

one_shot = generator(
    one_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("\n====================================")
print("📄 ONE-SHOT RESUME")
print("====================================\n")
print(one_shot)


📄 ONE-SHOT RESUME

You are an expert resume writer.

--- EXAMPLE START (SIMPLE FORMAT) ---
Software Engineer Resume
Name: Alex Johnson | Email: alex.j@mail.com
Summary: Software Engineer with 3 years of experience in full-stack development.
Skills: Java, Spring Boot, Docker, Kubernetes
Experience: Engineer at TechCo (2021–2023). Managed deployment pipelines.
Education: B.Tech in Computer Science
--- EXAMPLE END ---

TASK: Now, create a complete professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Include: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the new resume **exactly** like the simple example above.

<|assistant|>
**Professional Resume for Data Scientist**

**Summary:**
Alex Smith
Data Scientist

**Experience:**

**Data Scientist**
TechCo
2021–Present

*Managed and optimized data processing pipelines, ensuring efficient data analysis.*
*Developed predictive models using P

In [8]:
# Set 1 of FEW-SHOT PROMPTING
# =====================================================
few_shot_prompt = """You are an expert resume writer.

--- EXAMPLE 1: DATA ANALYST (3 years exp) ---
[Name]: Sarah Connor | [Phone]: 555-0001
Summary: Analyst with 3 years experience.
Skills: Excel, Python, PowerBI, SQL.
Experience: Analyst at FinCorp (2021-2024). Reduced reporting errors by 12%.
Education: BS in Economics.

--- EXAMPLE 2: SOFTWARE DEVELOPER (4 years exp) ---
[Name]: Michael B. | [Phone]: 555-0002
Summary: Developer with 4 years experience.
Skills: C++, Java, AWS, Docker.
Experience: Developer at Tech Giant (2020-2024). Accelerated deployment by 20%.
Education: MS in Computer Science.
--- END OF EXAMPLES ---

TASK: Create a professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Required Sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, Education.
Ensure the output is **one single, coherent resume** with clear headings, using the detailed style of the examples.
"""

few_shot = generator(
    few_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("\n====================================")
print("📄 FEW-SHOT RESUME")
print("====================================\n")
print(few_shot)


📄 FEW-SHOT RESUME

You are an expert resume writer.

--- EXAMPLE 1: DATA ANALYST (3 years exp) ---
[Name]: Sarah Connor | [Phone]: 555-0001
Summary: Analyst with 3 years experience.
Skills: Excel, Python, PowerBI, SQL.
Experience: Analyst at FinCorp (2021-2024). Reduced reporting errors by 12%.
Education: BS in Economics.

--- EXAMPLE 2: SOFTWARE DEVELOPER (4 years exp) ---
[Name]: Michael B. | [Phone]: 555-0002
Summary: Developer with 4 years experience.
Skills: C++, Java, AWS, Docker.
Experience: Developer at Tech Giant (2020-2024). Accelerated deployment by 20%.
Education: MS in Computer Science.
--- END OF EXAMPLES ---

TASK: Create a professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Required Sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, Education.
Ensure the output is **one single, coherent resume** with clear headings, using the detailed style of the examples.
<|assistant|>
**Profes

In [9]:
MAX_TOKENS = 500
TEMPERATURE = 0.6
TOP_P = 0.9

In [10]:
# Set 2 of ZERO-SHOT PROMPT
# =====================================================
zero_shot_prompt = """You are an expert resume writer.
Create a complete, single-page professional resume for a Data Scientist with 2 years of experience.
**Invent a generic name, contact details, company name, and specific achievements.**
Include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the final output ONLY as a resume, with clear headings and bullet points.
**DO NOT include any introductory text, invented job descriptions, or notes before or after the resume.**
"""

# Output variable name: zero_shot
zero_shot = generator(
    zero_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("====================================")
print("📄 ZERO-SHOT RESUME")
print("====================================\n")
print(zero_shot)

📄 ZERO-SHOT RESUME

You are an expert resume writer.
Create a complete, single-page professional resume for a Data Scientist with 2 years of experience.
**Invent a generic name, contact details, company name, and specific achievements.**
Include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the final output ONLY as a resume, with clear headings and bullet points.
**DO NOT include any introductory text, invented job descriptions, or notes before or after the resume.**
<|assistant|>
# Professional Resume

**Summary:**

Experienced Data Scientist with 2 years of expertise in leveraging Python, Machine Learning, SQL, and Statistics to drive data-driven insights and solutions. Proven track record in developing predictive models and optimizing data workflows, resulting in significant improvements in efficiency and accuracy.

**Skills:**

- **Python:** Proficient in using Python for data analysis, data visualization, and m

In [11]:
# Set 2 of ONE-SHOT PROMPT
# =====================================================
one_shot_prompt = """You are an expert resume writer.

--- EXAMPLE START (SIMPLE FORMAT) ---
Software Engineer Resume
Name: Alex Johnson | Email: alex.j@mail.com
Summary: Software Engineer with 3 years of experience in full-stack development.
Skills: Java, Spring Boot, Docker, Kubernetes
Experience: Engineer at TechCo (2021–2023). Managed deployment pipelines.
Education: B.Tech in Computer Science
--- EXAMPLE END ---

TASK: Now, create a complete professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Include: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the new resume **exactly** like the simple example above.
"""

# Output variable name: one_shot
one_shot = generator(
    one_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("====================================")
print("📄ONE-SHOT RESUME")
print("====================================\n")
print(one_shot)

📄ONE-SHOT RESUME

You are an expert resume writer.

--- EXAMPLE START (SIMPLE FORMAT) ---
Software Engineer Resume
Name: Alex Johnson | Email: alex.j@mail.com
Summary: Software Engineer with 3 years of experience in full-stack development.
Skills: Java, Spring Boot, Docker, Kubernetes
Experience: Engineer at TechCo (2021–2023). Managed deployment pipelines.
Education: B.Tech in Computer Science
--- EXAMPLE END ---

TASK: Now, create a complete professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Include: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
Format the new resume **exactly** like the simple example above.

<|assistant|>
# Data Scientist Resume

## Summary

Highly skilled and dedicated Data Scientist with 2 years of experience in leveraging advanced analytics and machine learning to drive impactful insights and solutions. Proven ability to translate complex data challenges into ac

In [12]:
# Set 2 of FEW-SHOT PROMPT
# =====================================================
few_shot_prompt = """You are an expert resume writer.

--- EXAMPLES (KEYWORD-ONLY) ---
RESUME 1: Role: Software Engineer | Skills: Java, Spring Boot, Docker
RESUME 2: Role: AI Researcher | Skills: PyTorch, NLP, Transformers
--- END OF EXAMPLES ---

TASK: Create a professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Required Sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, Education.
Ensure the output is **one single, coherent resume** with clear headings.
"""

# Output variable name: few_shot
few_shot = generator(
    few_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("\n====================================")
print("📄FEW-SHOT RESUME")
print("====================================\n")
print(few_shot)


📄FEW-SHOT RESUME

You are an expert resume writer.

--- EXAMPLES (KEYWORD-ONLY) ---
RESUME 1: Role: Software Engineer | Skills: Java, Spring Boot, Docker
RESUME 2: Role: AI Researcher | Skills: PyTorch, NLP, Transformers
--- END OF EXAMPLES ---

TASK: Create a professional resume for a Data Scientist with 2 years experience.
**Invent all personal and company details.**
Required Sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, Education.
Ensure the output is **one single, coherent resume** with clear headings.
<|assistant|>
```markdown
--- PROFESSIONAL RESUME TEMPLATE ---

**SUMMARY**
- Experienced Data Scientist with 2 years of expertise in Python, Machine Learning, SQL, and Statistics.
- Proficient in leveraging Python and R for data analysis and modeling.
- Strong problem-solving skills and a keen interest in advancing analytical capabilities.

**SKILLS**
- Python: Proficiency in both Data Science and Machine Learning frameworks.
- Machine Learning:

In [13]:
# Set 3 of ZERO-SHOT PROMPT
# =====================================================
zero_shot_prompt = """You are an expert resume writer. Your SOLE task is to create a professional resume.
You MUST invent a generic name, contact details, company name, and specific achievements.
You MUST only include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
You MUST output ONLY the resume content itself. DO NOT include any introductory sentences, notes, or commentary before or after the resume.

TASK: Create a professional resume for a Data Scientist with 2 years of experience."""

# Output variable name: zero_shot
zero_shot = generator(
    zero_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("====================================")
print("📄 ZERO-SHOT RESUME")
print("====================================\n")
print(zero_shot)

📄 ZERO-SHOT RESUME

You are an expert resume writer. Your SOLE task is to create a professional resume.
You MUST invent a generic name, contact details, company name, and specific achievements.
You MUST only include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
You MUST output ONLY the resume content itself. DO NOT include any introductory sentences, notes, or commentary before or after the resume.

TASK: Create a professional resume for a Data Scientist with 2 years of experience.
<|assistant|>
**Summary:**

**Experience:**

- **Data Scientist:** [Company Name], [City, State] [Duration]

- **Data Analyst:** [Company Name], [City, State] [Duration]

**Skills:**

- **Python:** Proficiency in Python for data analysis and machine learning.
- **Machine Learning:** Expertise in developing predictive models using Python and R.
- **SQL:** Strong command of SQL for database management and data retrieval.
- **Statistics:** Expertis

In [14]:
# set 3 of ONE-SHOT PROMPT
# =====================================================
one_shot_prompt = """You are an expert resume writer. Your SOLE task is to create a professional resume.
You MUST invent all names, contacts, company names, and specific achievements.
You MUST only include these exact sections: Summary, Skills, Experience, and Education.
You MUST output ONLY the resume content itself. DO NOT include any introductory sentences, notes, or commentary before or after the resume.

--- EXAMPLE START (SOFTWARE ENGINEER) ---
Name: Alex Johnson | Email: alex.j@mail.com | Phone: 555-0001
Summary: Software Engineer with 3 years of experience in full-stack development.
Skills: Java, Spring Boot, Docker, Kubernetes
Experience: Engineer at TechCo (2021–2024). Managed deployment pipelines and accelerated feature rollout by 30%.
Education: B.Tech in Computer Science from State University.
--- EXAMPLE END ---

TASK: Now, create a complete professional resume for a Data Scientist with 2 years experience.
"""

# Output variable name: one_shot
one_shot = generator(
    one_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("====================================")
print("📄ONE-SHOT RESUME")
print("====================================\n")
print(one_shot)

📄ONE-SHOT RESUME

You are an expert resume writer. Your SOLE task is to create a professional resume.
You MUST invent all names, contacts, company names, and specific achievements.
You MUST only include these exact sections: Summary, Skills, Experience, and Education.
You MUST output ONLY the resume content itself. DO NOT include any introductory sentences, notes, or commentary before or after the resume.

--- EXAMPLE START (SOFTWARE ENGINEER) ---
Name: Alex Johnson | Email: alex.j@mail.com | Phone: 555-0001
Summary: Software Engineer with 3 years of experience in full-stack development.
Skills: Java, Spring Boot, Docker, Kubernetes
Experience: Engineer at TechCo (2021–2024). Managed deployment pipelines and accelerated feature rollout by 30%.
Education: B.Tech in Computer Science from State University.
--- EXAMPLE END ---

TASK: Now, create a complete professional resume for a Data Scientist with 2 years experience.
<|assistant|>
**Resume**

**Summary**
Alex Smith | Data Scientist | [

In [16]:
# Set 3 of FEW-SHOT PROMPT
# =====================================================
few_shot_prompt = """You are an expert resume writer. Your SOLE task is to create a professional resume.
You MUST invent all names, contacts, company names, and specific achievements.
You MUST only include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
You MUST output ONLY the resume content itself. DO NOT include any introductory sentences, notes, or commentary before or after the resume.

--- EXAMPLE 1 (DATA ANALYST) ---
Name: Sarah Lin | Email: sarah.lin@mail.com | Phone: 555-4321
Summary: Data Analyst with 3 years of experience specializing in SQL and business intelligence.
Skills: SQL, Excel, Python, Tableau, Reporting
Experience: Analyst at FinServe (2021-2024). Increased reporting efficiency by 15% through SQL query optimization.
Education: BS in Economics from State University.

--- EXAMPLE 2 (SOFTWARE ENGINEER) ---
Name: Michael Chen | Email: mchen@dev.net | Phone: 555-9876
Summary: Software Developer with 4 years of experience in cloud-native applications.
Skills: Python, Java, AWS, Docker, Kubernetes
Experience: Developer at TechGiant (2020-2024). Reduced cloud hosting costs by 20% by refactoring core services.
Education: MS in Computer Science from Engineering Institute.

--- END OF EXAMPLES ---

FINAL OUTPUT RESUME: Data Scientist, 2 years experience."""

# Output variable name: few_shot
few_shot = generator(
    few_shot_prompt,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P
)[0]["generated_text"]

print("\n====================================")
print("📄FEW-SHOT RESUME")
print("====================================\n")
print(few_shot)


📄FEW-SHOT RESUME

You are an expert resume writer. Your SOLE task is to create a professional resume.
You MUST invent all names, contacts, company names, and specific achievements.
You MUST only include these exact sections: Summary, Skills (Python, Machine Learning, SQL, Statistics), Experience, and Education.
You MUST output ONLY the resume content itself. DO NOT include any introductory sentences, notes, or commentary before or after the resume.

--- EXAMPLE 1 (DATA ANALYST) ---
Name: Sarah Lin | Email: sarah.lin@mail.com | Phone: 555-4321
Summary: Data Analyst with 3 years of experience specializing in SQL and business intelligence.
Skills: SQL, Excel, Python, Tableau, Reporting
Experience: Analyst at FinServe (2021-2024). Increased reporting efficiency by 15% through SQL query optimization.
Education: BS in Economics from State University.

--- EXAMPLE 2 (SOFTWARE ENGINEER) ---
Name: Michael Chen | Email: mchen@dev.net | Phone: 555-9876
Summary: Software Developer with 4 years of